# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 11** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez </br>
**Team**: Par de Dos  
**Members**: Diego Orozco and Aarón Ortega

### FindSpark

In [1]:
import findspark
findspark.init()

### Connection with SparkCluster

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab11_ParDeDos") \
    .master("spark://368ad5a83fd7:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 19:51:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Schema and dataframe creation

In [6]:
from team_ParDeDos.spark_utils import SparkUtils as SpU

# Schema definition
schema = SpU.generate_schema([
  ("Id", "integer"),
  ("SepalLengthCm", "float"),
  ("SepalWidthCm", "float"),
  ("PetalLengthCm", "float"),
  ("PetalWidthCm", "float"),
  ("Species", "string")
])

# Dataframe creation
iris_df = spark \
          .read \
          .schema(schema) \
          .option("header", "true") \
          .option("mode", "dropMalformed") \
          .csv("/home/jovyan/notebooks/data/iris.csv")

### Label column to numeric values

In [8]:
from pyspark.ml.feature import StringIndexer

label_indexer = StringIndexer(inputCol="Species", outputCol="label")
data = label_indexer.fit(iris_df).transform(iris_df)

In [9]:
data.show()

+---+-------------+------------+-------------+------------+-----------+-----+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|
+---+-------------+------------+-------------+------------+-----------+-----+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|  0.0|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|  0.0|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|  0.0|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|  0.0|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|  0.0|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|  0.0|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|  0.0|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|  0.0|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-

### Assemble the features into a single vector column

In [10]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["Id", "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features")
data_with_features = assembler.transform(data).select("label", "features")

### Split the data into training and test sets

In [11]:
trained_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [12]:
print("Original Dataset")
data_with_features.show(5, truncate=False)

print("Trained Dataset")
trained_df.show(5, truncate=False)

Original Dataset
+-----+--------------------------------------------------------------------------------+
|label|features                                                                        |
+-----+--------------------------------------------------------------------------------+
|0.0  |[1.0,5.099999904632568,3.5,1.399999976158142,0.20000000298023224]               |
|0.0  |[2.0,4.900000095367432,3.0,1.399999976158142,0.20000000298023224]               |
|0.0  |[3.0,4.699999809265137,3.200000047683716,1.2999999523162842,0.20000000298023224]|
|0.0  |[4.0,4.599999904632568,3.0999999046325684,1.5,0.20000000298023224]              |
|0.0  |[5.0,5.0,3.5999999046325684,1.399999976158142,0.20000000298023224]              |
+-----+--------------------------------------------------------------------------------+
only showing top 5 rows

Trained Dataset
+-----+--------------------------------------------------------------------------------+
|label|features                                     

### Create a decision tree model

In [13]:
from pyspark.ml.classification import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(labelCol="label", featuresCol="features")

### Training

In [14]:
decision_tree_model = decision_tree.fit(trained_df)

print("Decision Tree model summary:{0}".format(decision_tree_model.toDebugString))

Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_3869034f651c, depth=3, numNodes=7, numClasses=3, numFeatures=5
  If (feature 3 <= 2.350000023841858)
   Predict: 0.0
  Else (feature 3 > 2.350000023841858)
   If (feature 0 <= 102.5)
    If (feature 4 <= 1.8499999642372131)
     Predict: 1.0
    Else (feature 4 > 1.8499999642372131)
     Predict: 2.0
   Else (feature 0 > 102.5)
    Predict: 2.0



### Predictions

In [16]:
predictions_dt = decision_tree_model.transform(test_df)

predictions_dt.select("features", "prediction").show(5, truncate=False)

+---------------------------------------------------------------------------------+----------+
|features                                                                         |prediction|
+---------------------------------------------------------------------------------+----------+
|[25.0,4.800000190734863,3.4000000953674316,1.899999976158142,0.20000000298023224]|0.0       |
|[26.0,5.0,3.0,1.600000023841858,0.20000000298023224]                             |0.0       |
|[33.0,5.199999809265137,4.099999904632568,1.5,0.10000000149011612]               |0.0       |
|[38.0,4.900000095367432,3.0999999046325684,1.5,0.10000000149011612]              |0.0       |
|[45.0,5.099999904632568,3.799999952316284,1.899999976158142,0.4000000059604645]  |0.0       |
+---------------------------------------------------------------------------------+----------+
only showing top 5 rows



### SVM Examples

In [18]:
from pyspark.ml.classification import LinearSVC, OneVsRest

lsvc = LinearSVC(maxIter=10, regParam=0.01)
ovr = OneVsRest(classifier=lsvc)

### Training

In [19]:
ovr_model = ovr.fit(trained_df)

25/05/04 20:12:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/04 20:12:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


### Predictions

In [20]:
predictions_svm = decision_tree_model.transform(test_df)

predictions_svm.select("features", "prediction").show(5, truncate=False)

+---------------------------------------------------------------------------------+----------+
|features                                                                         |prediction|
+---------------------------------------------------------------------------------+----------+
|[25.0,4.800000190734863,3.4000000953674316,1.899999976158142,0.20000000298023224]|0.0       |
|[26.0,5.0,3.0,1.600000023841858,0.20000000298023224]                             |0.0       |
|[33.0,5.199999809265137,4.099999904632568,1.5,0.10000000149011612]               |0.0       |
|[38.0,4.900000095367432,3.0999999046325684,1.5,0.10000000149011612]              |0.0       |
|[45.0,5.099999904632568,3.799999952316284,1.899999976158142,0.4000000059604645]  |0.0       |
+---------------------------------------------------------------------------------+----------+
only showing top 5 rows



### Model Testing

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_dt = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy_dt = evaluator_dt.evaluate(predictions_dt, {evaluator_dt.metricName: "accuracy"})
precision_dt = evaluator_dt.evaluate(predictions_dt,{evaluator_dt.metricName: "weightedPrecision"})
recall_dt = evaluator_dt.evaluate(predictions_dt,{evaluator_dt.metricName: "weightedRecall"})
f1_dt = evaluator_dt.evaluate(predictions_dt,{evaluator_dt.metricName: "f1"})

evaluator_svm = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy_svm = evaluator_svm.evaluate(predictions_svm, {evaluator_dt.metricName: "accuracy"})
precision_svm = evaluator_svm.evaluate(predictions_svm,{evaluator_dt.metricName: "weightedPrecision"})
recall_svm = evaluator_svm.evaluate(predictions_svm,{evaluator_dt.metricName: "weightedRecall"})
f1_svm = evaluator_svm.evaluate(predictions_svm,{evaluator_dt.metricName: "f1"})

In [26]:
print("Evaluation of Decision Tree")
print(f"Accuracy: {accuracy_dt:.4f}")
print(f"Precision: {precision_dt:.4f}")
print(f"Recall: {recall_dt:.4f}")
print(f"F1 Score: {f1_dt:.4f}")
print("Evaluation of Single Vector Machine")
print(f"Accuracy: {accuracy_svm:.4f}")
print(f"Precision: {precision_svm:.4f}")
print(f"Recall: {recall_svm:.4f}")
print(f"F1 Score: {f1_svm:.4f}")

Evaluation of Decision Tree
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Evaluation of Single Vector Machine
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


In [27]:
sc.stop()